In [210]:
import os
import pandas as pd
import datetime as dt
import logging
import numpy as np
from numpy import random
import gensim
import nltk
nltk.download('all')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cross_validation import train_test



[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/pranay/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package

ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [211]:
flares = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA"]


rindi = pd.read_csv('rindiaeda.csv')
rindi.head()

,title,url,comments,flair,score,id,author,body,timestamp
0,Will donate thrice the number of upvotes (amou...,https://www.reddit.com/r/india/comments/981o7s...,832,[R]eddiquette,19701,981o7s,TheBeginnerr,>**Note**: If you want to know what this is al...,2018-08-18 01:32:17
1,Fuck all Religion,https://www.reddit.com/r/india/comments/f9outu...,4198,Politics,17869,f9outu,pking3,"Fuck all religion. Fuck Hindusim, fuck Islam, ...",2020-02-26 19:40:49
2,Megathread: India-Pakistan border skirmish,https://www.reddit.com/r/india/comments/avafxp...,6913,[R]eddiquette,10205,avafxp,doc_two_thirty,There is a lot of news and speculation coming ...,2019-02-27 20:26:35
3,"Help, ideas needed. HLL poisoned my village wi...",https://www.reddit.com/r/india/comments/38heyu...,257,[R]eddiquette,8050,38heyu,UnileverCancerFactor,"I don't want to leave this be, and need some h...",2015-06-04 19:24:17
4,My friend passed away and TOI made a mockery o...,https://www.reddit.com/r/india/comments/4m5l2k...,241,[R]eddiquette,6561,4m5l2k,ForPranitaThrowaway,http://timesofindia.indiatimes.com/videos/news...,2016-06-02 18:30:32


In [212]:

def get_date(created):
    return dt.datetime.fromtimestamp(created)

def string_form(value):
    return str(value)

def clean_text(text):
     from sklearn.linear_model import SGDClassifier
     text = BeautifulSoup(text, "lxml").text
     text = text.lower()
     text = REPLACE_BY_SPACE_RE.sub(' ', text)
     text = BAD_SYMBOLS_RE.sub('', text)
     text = ' '.join(word for word in text.split() if word not in STOPWORDS)
     return text


In [213]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))



In [214]:
rindi['title'] = rindi['title'].apply(string_form)
rindi['body'] = rindi['body'].apply(string_form)
rindi['comments'] = rindi['comments'].apply(string_form)

rindi['title'] = rindi['title'].apply(clean_text)
rindi['body'] = rindi['body'].apply(clean_text)
rindi['comments'] = rindi['comments'].apply(clean_text)

feature_combine = rindi["title"] + rindi["comments"] + rindi["url"]
rindi = rindi.assign(feature_combine = feature_combine)

In [215]:
def nb_classifier(X_train, X_test, y_train, y_test):
  
  from sklearn.naive_bayes import MultinomialNB


  nb = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB()),
                ])
  nb.fit(X_train, y_train)

  y_pred = nb.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  #print(classification_report(y_test, y_pred,target_names=flares))


In [216]:
def linear_svm(X_train, X_test, y_train, y_test):
  
  from sklearn.linear_model import SGDClassifier

  sgd = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
                 ])
  sgd.fit(X_train, y_train)

  y_pred = sgd.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  #print(classification_report(y_test, y_pred,target_names=flairs))

In [217]:

def logisticreg(X_train, X_test, y_train, y_test):

    from sklearn.linear_model import LogisticRegression

    logreg = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                 ])
    logreg.fit(X_train, y_train)

    y_pred = logreg.predict(X_test)

    print('accuracy %s' % accuracy_score(y_pred, y_test))
   # print(classification_report(y_test, y_pred,target_names=flairs))


In [218]:

def randomforest(X_train, X_test, y_train, y_test):
  
  from sklearn.ensemble import RandomForestClassifier
  
  ranfor = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', RandomForestClassifier(n_estimators = 1000, random_state = 42)),
                 ])
  ranfor.fit(X_train, y_train)

  y_pred = ranfor.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  #print(classification_report(y_test, y_pred,target_names=flairs))


In [219]:

def mlpclassifier(X_train, X_test, y_train, y_test):
  
  from sklearn.neural_network import MLPClassifier
  
  mlp = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', MLPClassifier(hidden_layer_sizes=(30,30,30))),
                 ])
  mlp.fit(X_train, y_train)

  y_pred = mlp.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  #print(classification_report(y_test, y_pred,target_names=flairs))

In [220]:
def train_test(X,y):
 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

  print("Results of Naive Bayes Classifier")
  nb_classifier(X_train, X_test, y_train, y_test)
  print("Results of Linear Support Vector Machine")
  linear_svm(X_train, X_test, y_train, y_test)
  print("Results of Logistic Regression")
  logisticreg(X_train, X_test, y_train, y_test)
  print("Results of Random Forest")
  randomforest(X_train, X_test, y_train, y_test)
  print("Results of MLP Classifier")
 # mlpclassifier(X_train, X_test, y_train, y_test)

In [221]:
cat = rindi.flair

V = rindi.feature_combine
W = rindi.comments
X = rindi.title
Y = rindi.body
Z = rindi.url

print("Flair Detection using Title as Feature:")
print("\n")
train_test(X,cat)
print("\n")
print("Flair Detection using Body as Feature:")
print("\n")
train_test(Y,cat)
print("\n")
print("Flair Detection using URL as Feature:")
print("\n")
train_test(Z,cat)
print("\n")
print("Flair Detection using Comments as Feature:")
print("\n")
train_test(V,cat)


Flair Detection using Title as Feature:


Results of Naive Bayes Classifier
accuracy 0.2631578947368421
Results of Linear Support Vector Machine
accuracy 0.21052631578947367
Results of Logistic Regression
accuracy 0.2631578947368421
Results of Random Forest
accuracy 0.2631578947368421
Results of MLP Classifier


Flair Detection using Body as Feature:


Results of Naive Bayes Classifier
accuracy 0.2631578947368421
Results of Linear Support Vector Machine
accuracy 0.47368421052631576
Results of Logistic Regression
accuracy 0.5263157894736842
Results of Random Forest
accuracy 0.15789473684210525
Results of MLP Classifier


Flair Detection using URL as Feature:


Results of Naive Bayes Classifier
accuracy 0.2631578947368421
Results of Linear Support Vector Machine
accuracy 0.2631578947368421
Results of Logistic Regression
accuracy 0.2631578947368421
Results of Random Forest
accuracy 0.2631578947368421
Results of MLP Classifier


Flair Detection using Comments as Feature:


Results of Naive